In [ ]:
import math

# Third-party
import astropy.units as u
from astropy.constants import G
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
import numpy as np
%matplotlib inline
import scipy.optimize as so
from scipy.integrate import quad
from scipy.special import kn

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

import biff.scf as bscf

In [ ]:
def wz2012_helper(x, y, z, x0, y0, z0, q):
    r1 = (((x/x0)**2 + (y/y0)**2)**2 + (z/z0)**4)**0.25
    r2_2 = (q**2*(x**2+y**2) + z**2) / z0**2
    return r1, r2_2
    
def wz2012_density(x, y, z, x0, y0, z0, q): 
    r1, r2_2 = wz2012_helper(x, y, z, x0, y0, z0, q)
    r2 = math.sqrt(r2_2)
    return math.exp(-r1**2/2.) + r2**(-1.85)*math.exp(-r2)

def wz2012_density_arr(x, y, z, x0, y0, z0, q):
    r1, r2_2 = wz2012_helper(x, y, z, x0, y0, z0, q)
    r2 = np.sqrt(r2_2)
    return np.exp(-r1**2/2.) + r2**(-1.85)*np.exp(-r2)
    

# ---
def cao2013_density(x, y, z, x0, y0, z0):
    rr = (((x/x0)**2 + (y/y0)**2)**2 + (z/z0)**4)**0.25
    return kn(0, rr)

density = wz2012_density
density_arr = wz2012_density_arr
args = (1.49, 0.58, 0.4, 0.6)

# density = cao2013_density
# args = (0.67, 0.29, 0.27)

In [ ]:
def surf_dens_helper(z, q, density, axis=0):
    return density(q, z)

In [ ]:
xs = np.linspace(0.01, 10, 64)
surf_dens_x = np.zeros_like(xs)
surf_dens_y = np.zeros_like(xs)

_density_x = lambda q, z: density(q, 0, z, *args)
_density_y = lambda q, z: density(0, q, z, *args)

for i,x in enumerate(xs):
    surf_dens_x[i],_ = quad(surf_dens_helper, 
                            -10, 10,
                            args=(x, _density_x))

    surf_dens_y[i],_ = quad(surf_dens_helper, 
                            -10, 10,
                            args=(x, _density_y))

In [ ]:
plt.figure(figsize=(5,5))

xyz = np.zeros((3, 1024))
xyz[0] = np.logspace(-3, 1., xyz.shape[1])

analytic_dens_x = np.array([density(x, 0., 0., *args) for x in xyz[0]])
plt.semilogy(xyz[0], analytic_dens_x, marker='', linestyle='-')
plt.xlim(xyz[0].min(), xyz[0].max())
# plt.ylim(1E-6, 1E2)

plt.xlabel(r"$r$ [kpc]")
plt.ylabel(r"$\rho(r)$")

In [ ]:
plt.figure(figsize=(5,5))

plt.semilogy(xs, surf_dens_x, marker='', linestyle='-')
plt.semilogy(xs, surf_dens_y, marker='', linestyle='-')

plt.xlim(xs.min(), xs.max())
plt.ylim(1E-6, 1E2)

plt.xlabel(r"$R$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

In [ ]:
def density_on_grid(density, grid_lim=(-10,10), ngrid=128, args=()):
    grid = np.linspace(grid_lim[0], grid_lim[1], ngrid)
    xyz = np.vstack(map(np.ravel, np.meshgrid(grid,grid,grid)))

    val = np.zeros((ngrid*ngrid*ngrid,))
    val = density_arr(xyz[0], xyz[1], xyz[2], *args)
    val[np.isnan(val)] = val[np.isfinite(val)].max()
    
    gridx = xyz[0].reshape(ngrid,ngrid,ngrid)[:,:,0]
    gridy = xyz[1].reshape(ngrid,ngrid,ngrid)[:,:,0]
    
    return gridx, gridy, val

In [ ]:
ngrid = 128
xx,yy,dens3d = density_on_grid(density, ngrid=ngrid, args=args)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=2)
axes[0].contour(xx, yy, dens2d,
                levels=np.linspace(0.01, 10, 16))

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=0)
axes[1].contour(xx, yy, dens2d.T,
                levels=np.linspace(0.01, 10, 16))

---


In [ ]:
coeff = bscf.compute_coeffs(density, 
                            nmax=8, lmax=8, M=1., r_s=1., skip_odd=True,
                            args=args)
(S,Serr),(T,Terr) = coeff

In [ ]:
xyz = np.zeros((3, 1024))
xyz[0] = np.logspace(-3, 1., xyz.shape[1])

analytic_dens_x = np.array([density(x, 0., 0., *args) for x in xyz[0]])
biff_dens_x = bscf.SCFPotential(m=1., r_s=1., Snlm=S, Tnlm=np.zeros_like(S)).density(xyz)

xyz2 = np.zeros((3, 1024))
xyz2[1] = np.logspace(-3, 1., xyz2.shape[1])
analytic_dens_y = np.array([density(0., x, 0., *args) for x in xyz2[1]])
biff_dens_y = bscf.SCFPotential(m=1., r_s=1., Snlm=S, Tnlm=np.zeros_like(S)).density(xyz2)

fig, axes = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True)

axes[0].semilogy(xyz[0], analytic_dens_x, marker='None')
axes[0].semilogy(xyz[0], biff_dens_x, marker='None')

axes[1].semilogy(xyz2[1], analytic_dens_y, marker='None')
axes[1].semilogy(xyz2[1], biff_dens_y, marker='None')

In [ ]:
# np.save('n8_l8_wz2012.npy', S)

In [ ]:
def surf_dens_helper(z, R, pot, axis=0):
    q = np.array([0, 0, z])
    q[axis] = R
    return pot.density(q).value[0]

In [ ]:
pot = bscf.SCFPotential(m=1., r_s=1., Snlm=S, Tnlm=np.zeros_like(S))

In [ ]:
xs = np.linspace(0.01, 10, 32)
surf_dens_x = np.zeros_like(xs)

for i,x in enumerate(xs):
    _surf_dens,_ = quad(surf_dens_helper, 
                        -10, 10, 
                        args=(x, pot))
    surf_dens_x[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value

In [ ]:
plt.figure(figsize=(5,5))

plt.loglog(xs, surf_dens_x, marker='', linestyle='-')

plt.xlim(xs.min(), xs.max())
# plt.ylim(1E-6, 1E8)

plt.xlabel(r"$R$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

In [ ]:
grid = np.linspace(-10, 10, 256)

fig,axes = plt.subplots(1, 2, figsize=(12,6))
_ = pot.plot_contours(grid=(grid, grid, 0), ax=axes[0])
_ = pot.plot_contours(grid=(grid, 0, grid), ax=axes[1])
axes[0].axvline(5)
axes[0].axvline(-5)
axes[0].axhline(5)
axes[0].axhline(-5)

In [ ]:
ngrid = 128

_density = lambda x, y, z: pot.density(np.vstack((x,y,z)))
xx,yy,dens3d = density_on_grid(_density, ngrid=ngrid, args=args)

fig, axes = plt.subplots(1, 2, figsize=(10,5))

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=2)
axes[0].contour(xx, yy, dens2d,
                levels=np.linspace(0.01, 10, 16))

dens2d = dens3d.reshape(ngrid,ngrid,ngrid).sum(axis=0)
axes[1].contour(xx, yy, dens2d.T,
                levels=np.linspace(0.01, 10, 16))

In [ ]:
S.size